# Text modeling

Hello everyone! We're going ahead and creating models of our tweets data using `GenSim`. `GenSim` provides facilities for topic modeling and text classification that we will leverage later on.

In [220]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style as style
import pyLDAvis, pyLDAvis.sklearn
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation, PCA
pyLDAvis.enable_notebook()
style.use('fivethirtyeight')

### Data preparation
We're reusing our generated tweets from the previous section. 

In [ ]:
# Read cleaned texts
cleaned_tweets = pd.read_csv('out/cleaned_tweets.csv', index_col=None, header=0)
texts = cleaned_tweets['cleaned_text'].astype('U')
cleaned_tweets.head()

In [142]:
# Load existing vectorizers
dtm_v = pickle.load(open('out/dtm.pk', 'rb'))
ngram_v = pickle.load(open('out/ngram.pk', 'rb'))
tfidf_v = pickle.load(open('out/tfidf.pk', 'rb'))

In [143]:
# Load terms from existing vectorizers
dtm_terms = dtm_v.get_feature_names()
ngram_terms = ngram_v.get_feature_names()
tfidf_terms = tfidf_v.get_feature_names()

In [144]:
# Transform texts into representations
dtm = dtm_v.fit_transform(texts)
ngram = ngram_v.fit_transform(texts)
tfidf = tfidf_v.fit_transform(texts)

### Latent Dirichlet Allocation
`LDA` is used for `topic modeling` by classifying text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. A lot of math, but the important thing to remember is it can determine related terms that form `topics`.

We are going to user an additional package `pyLDAvis` to visualize our LDA results

In [212]:
def lda(texts):
    """
    Executes LDA algorithm to determine topics of the corpus
    """
    lda_model = LatentDirichletAllocation(
        n_components=5, max_iter=10,
        learning_method='online', 
        learning_offset=50., random_state=0)
    lda_model.fit_transform(texts)
    return lda_model

In [ ]:
# Execute LDA
dtm_lda = lda(dtm)
ngram_lda = lda(ngram)
tfidf_lda = lda(tfidf)

In [149]:
pyLDAvis.sklearn.prepare(dtm_lda, dtm, dtm_v)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.263653 -0.035424       1        1  23.487785
2     -0.034985  0.127147       2        1  22.455243
4     -0.047868  0.132613       3        1  19.090963
1     -0.091311 -0.032931       4        1  17.681870
0     -0.089490 -0.191405       5        1  17.284138, topic_info=     Category        Freq                 Term       Total  loglift  logprob
85    Default  142.000000                  amp  142.000000  30.0000  30.0000
881   Default  117.000000                 love  117.000000  29.0000  29.0000
1917  Default  108.000000           whodoylove  108.000000  28.0000  28.0000
119   Default  244.000000                apple  244.000000  27.0000  27.0000
968   Default   63.000000                 next   63.000000  26.0000  26.0000
1148  Default   73.000000              podcast   73.000000  25.0000  25.0000
1551  Default   59.000000                   st   59.000000  24.0000  24.0000
1908  Default   56.000000                whats   56.000000  23.0000  23.0000
1806  Default   55.000000              twitter   55.000000  22.0000  22.0000
981   Default   54.000000                 note   54.000000  21.0000  21.0000
1692  Default   53.000000      thesaracaershow   53.000000  20.0000  20.0000
1353  Default   53.000000  russiainvestigation   53.000000  19.0000  19.0000
375   Default   53.000000             cracking   53.000000  18.0000  18.0000
240   Default   39.000000                  bts   39.000000  17.0000  17.0000
964   Default   48.000000                  new   48.000000  16.0000  16.0000
776   Default   22.000000                https   22.000000  15.0000  15.0000
705   Default   34.000000                  get   34.000000  14.0000  14.0000
789   Default   23.000000                   im   23.000000  13.0000  13.0000
1883  Default   19.000000                watch   19.000000  12.0000  12.0000
1848  Default   22.000000                video   22.000000  11.0000  11.0000
603   Default   20.000000              episode   20.000000  10.0000  10.0000
896   Default   19.000000               making   19.000000   9.0000   9.0000
249   Default   19.000000                 call   19.000000   8.0000   8.0000
869   Default   19.000000               little   19.000000   7.0000   7.0000
948   Default   37.000000                music   37.000000   6.0000   6.0000
652   Default   19.000000             facetime   19.000000   5.0000   5.0000
1010  Default   14.000000             official   14.000000   4.0000   4.0000
1855  Default   17.000000           voicemails   17.000000   3.0000   3.0000
1547  Default   23.000000              spotify   23.000000   2.0000   2.0000
1809  Default   14.000000                  twt   14.000000   1.0000   1.0000
...       ...         ...                  ...         ...      ...      ...
448    Topic5    6.879416             deleting    7.564063   1.6605  -5.1842
833    Topic5    6.903204                 june    7.595806   1.6598  -5.1808
1041   Topic5    6.374846                order    7.061105   1.6531  -5.2604
897    Topic5    6.194847             mamazala    6.873355   1.6514  -5.2890
1584   Topic5    4.891021             straight    5.653024   1.6106  -5.5253
1310   Topic5    4.014281           retrospect    4.691095   1.5996  -5.7229
1213   Topic5    3.974627                  pty    4.648024   1.5989  -5.7328
1436   Topic5    3.973549                sharp    4.654849   1.5971  -5.7331
771    Topic5    3.974093              hottest    4.656222   1.5970  -5.7330
834    Topic5    3.973439                   ke    4.655946   1.5969  -5.7331
1211   Topic5    4.010339                proud    4.700559   1.5966  -5.7239
963    Topic5    9.196289                never   10.972693   1.5788  -4.8940
679    Topic5    3.158682             football    3.875935   1.5507  -5.9626
936    Topic5    4.499498                 mood    5.540923   1.5472  -5.6088
98     Topic5    5.789763            

In [151]:
pyLDAvis.sklearn.prepare(ngram_lda, ngram, ngram_v)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.272178  0.007900       1        1  25.727732
4     -0.079154  0.207405       2        1  22.127209
0     -0.061875 -0.079588       3        1  18.946692
1     -0.057713 -0.039708       4        1  18.167228
2     -0.073436 -0.096010       5        1  15.031139, topic_info=     Category       Freq                        Term      Total  loglift  \
388   Default  87.000000                    love amp  87.000000  30.0000   
29    Default  87.000000              amp whodoylove  87.000000  29.0000   
1391  Default  49.000000                  st podcast  49.000000  28.0000   
1680  Default  49.000000            twitter cracking  49.000000  27.0000   
681   Default  49.000000    next russiainvestigation  49.000000  26.0000   
701   Default  49.000000                note twitter  49.000000  25.0000   
979   Default  49.000000                podcast note  49.000000  24.0000   
1514  Default  49.000000       thesaracaershow whats  49.000000  23.0000   
1873  Default  49.000000                  whats next  49.000000  22.0000   
1218  Default  49.000000      russiainvestigation st  49.000000  21.0000   
54    Default  28.000000                 apple music  28.000000  20.0000   
71    Default  14.000000                 apple watch  14.000000  19.0000   
255   Default  15.000000                  get little  15.000000  18.0000   
1884  Default  15.000000                   whos call  15.000000  17.0000   
64    Default  15.000000                   apple sta  15.000000  16.0000   
133   Default  15.000000                 call missed  15.000000  15.0000   
382   Default  15.000000                little video  15.000000  14.0000   
1774  Default  15.000000              voicemails get  15.000000  13.0000   
881   Default  15.000000                 person whos  15.000000  12.0000   
222   Default  15.000000         facetime voicemails  15.000000  11.0000   
1392  Default  15.000000                  sta making  15.000000  10.0000   
1765  Default  15.000000                video person  15.000000   9.0000   
402   Default  15.000000             making facetime  15.000000   8.0000   
744   Default  13.000000  official releasedywaforbts  13.000000   7.0000   
1589  Default  13.000000            together tribute  13.000000   6.0000   
1987  Default  13.000000           yourwingsarmy bts  13.000000   5.0000   
1993  Default  13.000000           ywa yourwingsarmy  13.000000   4.0000   
1138  Default  13.000000       releasedywaforbts ywa  13.000000   3.0000   
1632  Default  13.000000                 tribute bts  13.000000   2.0000   
1989  Default  13.000000               youtube https  13.000000   1.0000   
...       ...        ...                         ...        ...      ...   
29     Topic5  87.243204              amp whodoylove  87.871114   1.8879   
1550   Topic5   2.997123              throwing apple   3.614645   1.7077   
418    Topic5   2.437317           marriage proposal   3.050821   1.6705   
30     Topic5   2.440961              ancient greece   3.058525   1.6695   
282    Topic5   2.438721             greece throwing   3.057746   1.6688   
72     Topic5   2.439862                 apple woman   3.060311   1.6685   
157    Topic5   2.436893         considered marriage   3.056897   1.6684   
1899   Topic5   2.434687            woman considered   3.055009   1.6681   
325    Topic5   1.992501            include hundreds   2.611865   1.6244   
309    Topic5   1.993309                hundreds fun   2.613167   1.6243   
1640   Topic5   1.990841              trivia include   2.610468   1.6241   
247    Topic5   1.988633             fun challenging   2.610525   1.6229   
1633   Topic5   1.987264               tricky trivia   2.609785   1.6225   
140    Topic5   1.989900       challenging questions   2.613658   1.6224   
735    Topic5   1.941717                   oder link   2.567850   1.6155   
1008   

In [152]:
pyLDAvis.sklearn.prepare(tfidf_lda, tfidf, tfidf_v)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.123965  0.088095       1        1  23.397299
2      0.008242 -0.056558       2        1  21.097428
1     -0.005153 -0.070197       3        1  20.906773
0     -0.131036  0.081250       4        1  19.121136
4      0.003982 -0.042590       5        1  15.477363, topic_info=     Category       Freq                 Term      Total  loglift  logprob
881   Default  45.000000                 love  45.000000  30.0000  30.0000
1917  Default  45.000000           whodoylove  45.000000  29.0000  29.0000
85    Default  45.000000                  amp  45.000000  28.0000  28.0000
968   Default  14.000000                 next  14.000000  27.0000  27.0000
1806  Default  13.000000              twitter  13.000000  26.0000  26.0000
981   Default  13.000000                 note  13.000000  25.0000  25.0000
1692  Default  13.000000      thesaracaershow  13.000000  24.0000  24.0000
1353  Default  13.000000  russiainvestigation  13.000000  23.0000  23.0000
375   Default  13.000000             cracking  13.000000  22.0000  22.0000
1908  Default  13.000000                whats  13.000000  21.0000  21.0000
1551  Default  13.000000                   st  13.000000  20.0000  20.0000
1148  Default  14.000000              podcast  14.000000  19.0000  19.0000
948   Default   9.000000                music   9.000000  18.0000  18.0000
240   Default   7.000000                  bts   7.000000  17.0000  17.0000
1883  Default   5.000000                watch   5.000000  16.0000  16.0000
119   Default  27.000000                apple  27.000000  15.0000  15.0000
776   Default   5.000000                https   5.000000  14.0000  14.0000
1848  Default   5.000000                video   5.000000  13.0000  13.0000
1095  Default   4.000000               person   4.000000  12.0000  12.0000
652   Default   4.000000             facetime   4.000000  11.0000  11.0000
869   Default   4.000000               little   4.000000  10.0000  10.0000
1855  Default   4.000000           voicemails   4.000000   9.0000   9.0000
1552  Default   4.000000                  sta   4.000000   8.0000   8.0000
1919  Default   4.000000                 whos   4.000000   7.0000   7.0000
929   Default   4.000000               missed   4.000000   6.0000   6.0000
249   Default   4.000000                 call   4.000000   5.0000   5.0000
1829  Default   4.000000                  use   4.000000   4.0000   4.0000
60    Default   4.000000                album   4.000000   3.0000   3.0000
287   Default   3.000000                chose   3.000000   2.0000   2.0000
1010  Default   3.000000             official   3.000000   1.0000   1.0000
...       ...        ...                  ...        ...      ...      ...
850    Topic5   1.209856           largescale   1.694771   1.5287  -5.7618
215    Topic5   0.983624              bidding   1.473071   1.4619  -5.9688
1198   Topic5   0.984006           production   1.475191   1.4609  -5.9684
880    Topic5   0.983432                loses   1.475064   1.4604  -5.9690
1595   Topic5   0.983384               studio   1.475200   1.4602  -5.9691
1093   Topic5   1.199672              perhaps   1.830449   1.4433  -5.7703
934    Topic5   0.898405               monkey   1.371775   1.4426  -6.0594
1829   Topic5   2.632512                  use   4.033602   1.4391  -4.9844
622    Topic5   0.923599                every   1.417248   1.4376  -6.0318
911    Topic5   0.827112            mcconnell   1.303986   1.4106  -6.1421
386    Topic5   0.817724               credit   1.295717   1.4055  -6.1535
989    Topic5   0.814359             nstalgic   1.295062   1.4019  -6.1577
1206   Topic5   0.797079             proposal   1.269762   1.4002  -6.1791
512    Topic5   0.796992            documents   1.271492   1.3987  -6.1792
297    Topic5   0.803087              classic   1.282186   1.3979  -6.1716
87     Topic5   0.798591              ancient   1.2770

### K-means clustering
We can find interesting groupings that come out of the data using `K-means`. This algorithm creates `clusters` of data by separating samples into `n` number of groups of equal variance that surround centroids. 

Because our text data representations have many dimensions (1000, 2000), it will be difficult to visualize this properly. However, we will be using dimensionality reduction techniques such as `Prinicipal Component Analysis` and `t-Distributed Stochastic Neighbor Embedding` to reduce our dimensions to two. Again, a lot of mathematics, but an important thing to remember is that both "combine" existing dimensions until only the desired number of dimensions is reached. 

In [214]:
def kmeans(texts, clusters=5):
    """
    Execute Kmeans algorithm to generate clusters
    """
    kmeans_model = KMeans(n_clusters=clusters, max_iter=1000)
    kmeans_model.fit(texts)
    kmeans_result = kmeans_model.predict(texts)
    return kmeans_result

In [216]:
def plot_kmeans(texts, clusters): 
    """
    Generates visualizations of clusters using Prinicipal Component Analysis
    and t-Distributed Stochastic Neighbor Embedding
    """
    pca = PCA(n_components=2).fit_transform(texts.todense())
    tsne = TSNE().fit_transform(PCA(n_components=50).fit_transform(texts.todense()))
    pca_a = pca[:, 0]
    pca_b = pca[:, 1]
    tsne_a = tsne[:, 0]
    tsne_b = tsne[:, 1]
    
    f, ax = plt.subplots(1, 2)
    sns.scatterplot(x=pca_a, y=pca_b, hue=clusters, ax=ax[0], )
    sns.scatterplot(x=tsne_a, y=tsne_b, hue=clusters, ax=ax[1], )

In [ ]:
dtm_kmeans = kmeans(dtm)
plot_kmeans(dtm, dtm_kmeans)

In [ ]:
ngram_kmeans = kmeans(ngram)
plot_kmeans(ngram, ngram_kmeans)

In [ ]:
tfidf_kmeans = kmeans(tfidf)
plot_kmeans(tfidf, tfidf_kmeans)